In [1]:
import torch
import gpytorch
from botorch.models import SingleTaskGP
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.models.transforms.outcome import Standardize
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_torch # fix Cholecky jitter error
from scipy.stats import norm
from sklearn.decomposition import PCA
import math 
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle
import os
import time

# config plot settings
plt.rcParams["font.size"] = 16

In [2]:
###
#  Load Data
###
file = h5py.File("targets_and_normalized_features.jld2", "r")
# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))
# simulation data (high fidelity)
y = torch.from_numpy(np.transpose(file["gcmc_y"][:]))
# associated simulation costs
cost = torch.from_numpy(np.transpose(file["gcmc_elapsed_time"][:]))

# total number of COFs in data set
nb_COFs = X.shape[0]

print("raw data - \n\tX:", X.shape)
print("\t\ty:", y.shape)
print("\t\tcost: ", cost.shape)    
    
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)
print("width:\n",torch.max(X, 0).values - torch.min(X, 0).values)

raw data - 
	X: torch.Size([608, 14])
		y: torch.Size([608])
		cost:  torch.Size([608])

Ensure features are normalized - 
max:
 tensor([0.7144, 0.4136, 0.4696, 0.6677, 0.9579, 0.8383, 0.3595, 0.3207, 0.9938,
        0.8242, 0.9692, 0.9869, 0.9868, 0.9762], dtype=torch.float64)
min:
 tensor([-0.2856, -0.5864, -0.5304, -0.3323, -0.0421, -0.1617, -0.6405, -0.6793,
        -0.0062, -0.1758, -0.0308, -0.0131, -0.0132, -0.0238],
       dtype=torch.float64)
width:
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000], dtype=torch.float64)


In [3]:
X_unsqueezed = X.unsqueeze(1)

In [ ]:
###
#  Load initializing COFs
###
init_cof_ids_file = pickle.load(open('search_results/initializing_cof_ids.pkl', 'rb'))
init_cof_ids = init_cof_ids_file['init_cof_ids']
init_cof_ids[:5]

## Helper Functions

#### Construct Initial Inputs

In [4]:
# # find COF closest to the center of feature space
# def get_initializing_COF(X):
#     # center of feature space
# #     feature_center = np.ones(X.shape[1]) * 0.5
#     data_center = np.array([X[:, i].mean() for i in range(X.size()[1])])
#     # max possible distance between normalized features
#     return np.argmin(np.linalg.norm(X - data_center, axis=1))

# # yields np.array([25, 494, 523])
# def diverse_set(X, train_size):
#     # initialize with one random point; pick others in a max diverse fashion
#     ids_train = [get_initializing_COF(X)]
#     # select remaining training points
#     for j in range(train_size - 1):
#         # for each point in data set, compute its min dist to training set
#         dist_to_train_set = np.linalg.norm(X - X[ids_train, None, :], axis=2)
#         assert np.shape(dist_to_train_set) == (len(ids_train), nb_COFs)
#         min_dist_to_a_training_pt = np.min(dist_to_train_set, axis=0)
#         assert np.size(min_dist_to_a_training_pt) == nb_COFs
        
#         # acquire point with max(min distance to train set) i.e. Furthest from train set
#         ids_train.append(np.argmax(min_dist_to_a_training_pt))
#     assert np.size(np.unique(ids_train)) == train_size # must be unique
#     return np.array(ids_train)

In [5]:
# construct feature matrix of acquired points
def build_X_train(ids_acquired):
    return X[ids_acquired, :]

# construct output vector for acquired points
def build_y_train(ids_acquired):
    return y[ids_acquired].unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(ids_acquired):
    return cost[ids_acquired]

In [6]:
# construct and fit GP model
def construct_and_fit_gp_model(X_train, y_train):      
    model = SingleTaskGP(X_train, y_train, outcome_transform=Standardize(m=1))
    mll   = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

#### Bayesian Algorithm

In [7]:
def run_Bayesian_optimization(nb_iterations, nb_COFs_initialization):
    assert nb_iterations > nb_COFs_initialization
    ###
    #  Initialize system
    ###
    # select initial COFs for training data
    ids_acquired = diverse_set(X, nb_COFs_initialization)
    
    ###
    #  itterate through remaining budget using BO
    ###
    for i in range(nb_COFs_initialization, nb_iterations):
        # construct training data (perform experiments)
        X_train = build_X_train(ids_acquired)
        y_train = build_y_train(ids_acquired)
        
        # fit GP surrogate model
        model = construct_and_fit_gp_model(X_train, y_train)
        
        # compute expected improvement acquisition function
        acq_fn   = ExpectedImprovement(model, best_f=y_train.max().item())
        acq_vals = acq_fn.forward(X.unsqueeze(1))
        
        # identify COF with highest acquisition value currently *not* in training set
        ids_sorted_by_aquisition = acq_vals.argsort(descending=True)
        for id_max in ids_sorted_by_aquisition:
            if not id_max.item() in ids_acquired:
                id_max_acq = id_max.item()
                break
                
        ###
        #  acquire this COF
        ###
        ids_acquired = np.concatenate((ids_acquired, [id_max_acq]))
        print("BO iteration ", i)
        print("\tacquired COF ", id_max_acq)
        print("\ty = ", y[id_max_acq].item())
    
    # check budget constraint is stisfied
    assert np.size(ids_acquired) == nb_iterations
    assert len(np.unique(ids_acquired)) == nb_iterations
    return ids_acquired

# Run BO

In [8]:
ids_acquired = diverse_set(X, 3)
ids_acquired

array([112, 522,  45])

In [9]:
nb_iterations = 75
nb_COFs_initialization = 3

ids_acquired = run_Bayesian_optimization(nb_iterations, nb_COFs_initialization)

y_acquired    = y[ids_acquired]
cost_acquired = build_cost(ids_acquired)

Iter 10/100: 2.5316478673813574
Iter 20/100: 1.5976805610155178
Iter 30/100: 1.5335137562067445
Iter 40/100: 1.5103225146173305
Iter 50/100: 1.4670837577792153
Iter 60/100: 1.4641026733095588
BO iteration  3
	acquired COF  521
	y =  15.766064256252303
Iter 10/100: 2.2821451376626243
Iter 20/100: 1.576745767814489
Iter 30/100: 1.516054702172481
Iter 40/100: 1.4776026924306396
Iter 50/100: 1.4418954556648251
Iter 60/100: 1.43763657751167
BO iteration  4
	acquired COF  523
	y =  14.017515356923804
Iter 10/100: 2.1182714695918206
Iter 20/100: 1.5406069636537971
Iter 30/100: 1.465887253647115
Iter 40/100: 1.4001012223371432
Iter 50/100: 1.3578769891536706
Iter 60/100: 1.3454021759424304
BO iteration  5
	acquired COF  524
	y =  6.895659242359536
Iter 10/100: 2.0287369141547815
Iter 20/100: 1.5417800785110842
Iter 30/100: 1.4659487823993385
Iter 40/100: 1.4034512338980265
Iter 50/100: 1.3717580593803287
Iter 60/100: 1.361105079632785
Iter 70/100: 1.3618015872844438
Iter 80/100: 1.360189181915

Iter 40/100: 1.2713704660939757
Iter 50/100: 1.170791985141234
BO iteration  42
	acquired COF  14
	y =  6.881047378855367
Iter 10/100: 1.5669156552054664
Iter 20/100: 1.4287022782510435
Iter 30/100: 1.3630345296394106
Iter 40/100: 1.2805961425355863
Iter 50/100: 1.2441196726971104
BO iteration  43
	acquired COF  515
	y =  6.237059454214897
Iter 10/100: 1.564915443037944
Iter 20/100: 1.4280230234141986
Iter 30/100: 1.3625088023252885
Iter 40/100: 1.2801281197458534
Iter 50/100: 1.246790077260442
BO iteration  44
	acquired COF  149
	y =  4.870990027657603
Iter 10/100: 1.5626046190166525
Iter 20/100: 1.4269298032094793
Iter 30/100: 1.3612435080037728
Iter 40/100: 1.2792252398340007
Iter 50/100: 1.2486154006203734
BO iteration  45
	acquired COF  10
	y =  6.473829589979741
Iter 10/100: 1.5607194918637448
Iter 20/100: 1.4263514887445374
Iter 30/100: 1.3610154534330434
Iter 40/100: 1.2800204331328113
Iter 50/100: 1.2505792123707398
BO iteration  46
	acquired COF  282
	y =  2.9106750650702327


In [10]:
# accounts for ratification step i.e. the extra step need to perform experiment and verify the property 
BO_iter_top_cof_acquired = np.argmax(y_acquired == max(y))
top_cof_acc_cost = sum(cost_acquired[:BO_iter_top_cof_acquired]).item() 

print("iteration we acquire top COF = ", BO_iter_top_cof_acquired.item())
print("accumulated cost up to observation of top COF = ", top_cof_acc_cost, " [min]")

iteration we acquire top COF =  35
accumulated cost up to observation of top COF =  6909.4041180729855  [min]


In [11]:
###
#  Store SFBO results
###
sfbo_res = dict({'ids_acquired': ids_acquired,
                 'y_acquired': y_acquired.detach().numpy(),
                 'cost_acquired': cost_acquired.detach().numpy(),
                 'nb_COFs_initialization': nb_COFs_initialization,
                 'BO_iter_top_cof_acquired': BO_iter_top_cof_acquired.item()
                })

with open('search_results/sfbo_results_with_EI.pkl', 'wb') as file:
    pickle.dump(sfbo_res, file)

# Random Search

In [12]:
nb_runs = 1000

rs_res = dict()
rs_res['ids_acquired'] = []
rs_res['cost_acquired'] = []

for r in range(nb_runs):
    rs_ids_acquired = np.random.choice(range(nb_COFs), replace=False, size=nb_iterations)
    rs_cost_acquired = build_cost(rs_ids_acquired)
    rs_res['ids_acquired'].append(rs_ids_acquired)
    rs_res['cost_acquired'].append(rs_cost_acquired)

In [13]:
def test_rs(rs_res):
    for a in np.random.choice(nb_runs, replace=False, size=7):
        # check that the lengths 
        assert len(rs_res['ids_acquired'][a]) == nb_iterations
        assert all([cof_id in range(nb_COFs) for cof_id in rs_res['ids_acquired'][a]])
    return

test_rs(rs_res)

In [14]:
# get y_max acquired up to iteration i for i = 1,2,...
def y_max(rs_ids_acquired):
    y_max_mu      = np.zeros(nb_iterations)
    y_max_sig_bot = np.zeros(nb_iterations)
    y_max_sig_top = np.zeros(nb_iterations)
    # element i of these will be y max at BO iter i
    
    for i in range(1, nb_iterations+1):
        # max value acquired up to this point
        y_maxes = np.array([max(y[rs_ids_acquired[r][:i]]) for r in range(nb_runs)])
        assert np.size(y_maxes) == nb_runs
        y_max_mu[i-1]      = np.mean(y_maxes)
        y_max_sig_bot[i-1] = np.std(y_maxes[y_maxes < y_max_mu[i-1]])
        y_max_sig_top[i-1] = np.std(y_maxes[y_maxes > y_max_mu[i-1]])
    return y_max_mu, y_max_sig_bot, y_max_sig_top

# rs_mean, rs_lower_bound, rs_upper_bound = y_max(rs_res)
y_rs_max_mu, y_rs_max_sig_bot, y_rs_max_sig_top = y_max(rs_res['ids_acquired'])

In [15]:
###
#  Store Random Search Results
###
random_search_results = dict({'ids_acquired': rs_res['ids_acquired'],
                             'y_rs_max_mu': y_rs_max_mu,
                             'y_rs_max_sig_bot': y_rs_max_sig_bot,
                             'y_rs_max_sig_top': y_rs_max_sig_top,
                             'cost_acquired': rs_res['cost_acquired']
                             })

with open('search_results/random_search_results.pkl', 'wb') as file:
    pickle.dump(random_search_results, file)